I used this notebook to extract movie data from imdb. I printed this data to 'info.csv' file and used it in the interface.

In [1]:
import pandas as pd
import numpy as np

import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint

In [18]:
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []
img=[]
genres=[]
text=[]

In [19]:
headers = {'Accept-Language': 'en-US, en;q=0.5'}

In [20]:
pages = np.arange(1, 1001, 50)
pages

array([  1,  51, 101, 151, 201, 251, 301, 351, 401, 451, 501, 551, 601,
       651, 701, 751, 801, 851, 901, 951])

In [5]:
for page in pages:
    # Getting the contents from the each url
    page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&start=' + str(page) + '&ref_=adv_nxt', headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Aiming the part of the html we want to get the information from
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
    # Waiting time between requests for a number between 2-10 seconds
    sleep(randint(2,10))
    
    for container in movie_div:
        # Scraping the movie's name
        name = container.h3.a.text
        titles.append(name)
        
        # Scraping the movie's year
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        
        # Scraping the movie's length
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        time.append(runtime)
        
        # Scraping the rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        # Scraping the metascore
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)


        muted_text_0 = container.find_all("p", {"class":  "text-muted"})[0]
        muted_text_1 = container.find_all("p", {"class":  "text-muted"})[1]
        
        genre=muted_text_0.find('span','genre').text
        genres.append(genre)

        summary=muted_text_1.text
        text.append(summary)

        imageDiv =  container.find("div", {"class": "lister-item-image float-left"})
        image = imageDiv.find("img", "loadlate")
        img.append(image.get('loadlate'))
        
        # Scraping votes and gross earnings
        nv = container.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses)

In [19]:
movies = pd.DataFrame({'movie':titles,
                       'year':years,
                       'time_minute':time,
                       'imdb_rating':imdb_ratings,
                       'metascore':metascores,
                       'vote':votes,
                       'gross_earning':us_gross,
                       'img_url':img,
                       'genres':genres,
                       'summary':text})
movies


,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning,img_url,genres,summary
0,Avatar: The Way of Water,(2022),192 min,7.9,67,"151,362",-,https://m.media-amazon.com/images/M/MV5BYjhiNj...,"\nAction, Adventure, Fantasy",\nJake Sully lives with his newfound family fo...
1,Avatar,(2009),162 min,7.9,83,"1,281,808",$760.51M,https://m.media-amazon.com/images/M/MV5BZDA0OG...,"\nAction, Adventure, Fantasy",\nA paraplegic Marine dispatched to the moon P...
2,The Banshees of Inisherin,(2022),114 min,7.9,87,"51,157",-,https://m.media-amazon.com/images/M/MV5BM2NlZD...,"\nComedy, Drama",\nTwo lifelong friends find themselves at an i...
3,Guillermo del Toro's Pinocchio,(2022),117 min,7.7,79,"50,199",-,https://m.media-amazon.com/images/M/MV5BNjBhZj...,"\nAnimation, Drama, Family",\nA father's wish magically brings a wooden bo...
4,Knives Out,(2019),130 min,7.9,82,"652,307",$165.36M,https://m.media-amazon.com/images/M/MV5BMGUwZj...,"\nComedy, Crime, Drama",\nA detective investigates the death of the pa...
...,...,...,...,...,...,...,...,...,...,...
995,Baby,(I) (2015),159 min,7.9,-,"58,237",-,https://m.media-amazon.com/images/M/MV5BYTdhNj...,"\nAction, Crime, Thriller",\nAn elite counter-intelligence unit learns of...
996,Special 26,(2013),144 min,8.0,-,"56,800",$1.08M,https://m.media-amazon.com/images/M/MV5BMTQ1ND...,"\nCrime, Drama, Thriller",\nA gang of con-men rob prominent rich busines...
997,Udaan,(2010),134 min,8.1,-,"45,421",$0.01M,https://m.media-amazon.com/images/M/MV5BNzgxMz...,\nDrama,"\nExpelled from his school, a 16-year old boy ..."
998,Sarfarosh,(1999),174 min,8.1,-,"25,982",-,https://m.media-amazon.com/images/M/MV5BZTljOT...,"\nAction, Drama, Thriller",\nAfter his brother is killed and father sever...


In [24]:
movies.to_csv('info.csv', encoding='utf-8', index=False)

In [35]:
df=pd.read_csv('info.csv')
df["movie"] = df['movie'].astype(str) +" "+ df["year"]
df=df.drop(['year'],axis=1)
df.to_csv('info.csv', encoding='utf-8', index=False)

In [2]:
df=pd.read_csv('info.csv')
df

,movie,time_minute,imdb_rating,metascore,vote,gross_earning,img_url,genres,summary
0,Avatar: The Way of Water (2022),192 min,7.9,67,"151,362",-,https://m.media-amazon.com/images/M/MV5BYjhiNj...,"\nAction, Adventure, Fantasy",\nJake Sully lives with his newfound family fo...
1,Avatar (2009),162 min,7.9,83,"1,281,808",$760.51M,https://m.media-amazon.com/images/M/MV5BZDA0OG...,"\nAction, Adventure, Fantasy",\nA paraplegic Marine dispatched to the moon P...
2,The Banshees of Inisherin (2022),114 min,7.9,87,"51,157",-,https://m.media-amazon.com/images/M/MV5BM2NlZD...,"\nComedy, Drama",\nTwo lifelong friends find themselves at an i...
3,Guillermo del Toro's Pinocchio (2022),117 min,7.7,79,"50,199",-,https://m.media-amazon.com/images/M/MV5BNjBhZj...,"\nAnimation, Drama, Family",\nA father's wish magically brings a wooden bo...
4,Knives Out (2019),130 min,7.9,82,"652,307",$165.36M,https://m.media-amazon.com/images/M/MV5BMGUwZj...,"\nComedy, Crime, Drama",\nA detective investigates the death of the pa...
...,...,...,...,...,...,...,...,...,...
995,Baby (I) (2015),159 min,7.9,-,"58,237",-,https://m.media-amazon.com/images/M/MV5BYTdhNj...,"\nAction, Crime, Thriller",\nAn elite counter-intelligence unit learns of...
996,Special 26 (2013),144 min,8.0,-,"56,800",$1.08M,https://m.media-amazon.com/images/M/MV5BMTQ1ND...,"\nCrime, Drama, Thriller",\nA gang of con-men rob prominent rich busines...
997,Udaan (2010),134 min,8.1,-,"45,421",$0.01M,https://m.media-amazon.com/images/M/MV5BNzgxMz...,\nDrama,"\nExpelled from his school, a 16-year old boy ..."
998,Sarfarosh (1999),174 min,8.1,-,"25,982",-,https://m.media-amazon.com/images/M/MV5BZTljOT...,"\nAction, Drama, Thriller",\nAfter his brother is killed and father sever...
